In [26]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math

In [42]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

In [43]:
ps.head()

,Date,User,Rig #,File,Ephys,Trans,Morph,Pilot,Pilot Details,cell type,...,Plate,Comments,Stacis call for reconstructions,Stacis call for class,Confidence,Bad dates,Notes.1,Unnamed: 65,Unnamed: 66,Unnamed: 67
0,160318,P5,NaN,Ndnf - brl_160318_01,x,NaN,n,NaN,NaN,tdt+,...,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,234626,NaN
1,160318,P5,NaN,Ndnf - brl_160318_02,x,NaN,n,NaN,NaN,tdt+,...,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,234626,NaN
2,160318,P5,NaN,Ndnf - brl_160318_03,x,NaN,y,NaN,NaN,tdt+,...,NaN,NaN,F,aspiny,NaN,NaN,NaN,NaN,234626,NaN
3,160318,P5,NaN,Ndnf - brl_160318_04,x,x,y,NaN,NaN,tdt+,...,NaN,NaN,F,aspiny,NaN,NaN,0,NaN,234626,NaN
4,160318,P5,NaN,Ndnf - brl_160318_05,x,NaN,n,NaN,NaN,tdt+,...,NaN,NaN,n,NaN,NaN,NaN,NaN,NaN,234626,NaN


In [44]:
ps = ps[["Date", "User", "Post patch?", "Notes", "PCR cycles", "SM_QC_PF", "Resolution index", "Bad dates"]]
ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps = ps[ps["Bad dates"] != 'x']
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["PCR cycles"] == 21]
ps = ps.drop('Bad dates', 1)
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
#ps["PCR cycles"].value_counts()
ps

,Date,User,Post patch?,Notes,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,21.0,pass,0.787397
4719,2017-04-19,P2,No-Seal,NaN,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,21.0,pass,0.322631
4722,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.322631
4723,2017-04-19,P2,No-Seal,NaN,21.0,pass,0.000000
4724,2017-04-19,P2,Nucleated,NaN,21.0,pass,NaN
4727,2017-04-19,P6,No-Seal,NaN,21.0,fail,NaN
4728,2017-04-19,P6,No-Seal,NaN,21.0,pass,0.000000


In [45]:
total = ps["Post patch?"].count()
total

1388

In [46]:
nuc = ps[ps["Post patch?"] == "Nucleated"]
nuc

,Date,User,Post patch?,Notes,PCR cycles,SM_QC_PF,Resolution index
4720,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.000000
4722,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.322631
4724,2017-04-19,P2,Nucleated,NaN,21.0,pass,NaN
4737,2017-04-20,P1,Nucleated,NaN,21.0,pass,0.601033
4746,2017-04-20,P1,Nucleated,NaN,21.0,pass,NaN
4758,2017-04-20,P2,Nucleated,NaN,21.0,pass,0.932314
4771,2017-04-21,P1,Nucleated,"Fluorescence in Pipette,Cell Dimmed",21.0,pass,0.781350
4775,2017-04-21,P2,Nucleated,Fluorescence in Pipette,21.0,pass,1.000000
4776,2017-04-21,P2,Nucleated,Fluorescence in Pipette,21.0,pass,0.877682
4784,2017-04-21,P2,Nucleated,NaN,21.0,pass,0.763111


In [47]:
nuc_count = nuc["Post patch?"].count()
nuc_count

607

In [53]:
nuc_percent = float(nuc_count)/total
nuc_percent

0.4373198847262248

In [54]:
no_p1_total = ps[ps["User"] != 'P1']
no_p1_total

,Date,User,Post patch?,Notes,PCR cycles,SM_QC_PF,Resolution index
4717,2017-04-19,P2,Outside-Out,NaN,21.0,fail,NaN
4718,2017-04-19,P2,Outside-Out,NaN,21.0,pass,0.787397
4719,2017-04-19,P2,No-Seal,NaN,21.0,fail,NaN
4720,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.000000
4721,2017-04-19,P2,Outside-Out,NaN,21.0,pass,0.322631
4722,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.322631
4723,2017-04-19,P2,No-Seal,NaN,21.0,pass,0.000000
4724,2017-04-19,P2,Nucleated,NaN,21.0,pass,NaN
4727,2017-04-19,P6,No-Seal,NaN,21.0,fail,NaN
4728,2017-04-19,P6,No-Seal,NaN,21.0,pass,0.000000


In [55]:
no_p1_nuc = no_p1_total[no_p1_total["Post patch?"] == "Nucleated"]
no_p1_nuc

,Date,User,Post patch?,Notes,PCR cycles,SM_QC_PF,Resolution index
4720,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.000000
4722,2017-04-19,P2,Nucleated,NaN,21.0,pass,0.322631
4724,2017-04-19,P2,Nucleated,NaN,21.0,pass,NaN
4758,2017-04-20,P2,Nucleated,NaN,21.0,pass,0.932314
4775,2017-04-21,P2,Nucleated,Fluorescence in Pipette,21.0,pass,1.000000
4776,2017-04-21,P2,Nucleated,Fluorescence in Pipette,21.0,pass,0.877682
4784,2017-04-21,P2,Nucleated,NaN,21.0,pass,0.763111
4807,2017-04-24,P2,Nucleated,NaN,21.0,pass,0.601033
4812,2017-04-24,P2,Nucleated,Fluorescence in Pipette One big bubble formed ...,21.0,pass,0.596520
4833,2017-04-25,P2,Nucleated,NaN,21.0,pass,0.875031


In [57]:
no_p1_count = no_p1_total["Post patch?"].count()
no_p1_count

842

In [59]:
no_p1_nuccount = no_p1_nuc["Post patch?"].count()
no_p1_nuccount

420

In [61]:
new_no_p1_percent = float(no_p1_nuccount)/no_p1_count
new_no_p1_percent

0.498812351543943